<a href="https://colab.research.google.com/github/mmarushika/sdc-lab/blob/main/RAGMedicalQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# ✅ STEP 1: Install Dependencies (including langchain-community)


# ✅ STEP 2: Load Sample Medical Documents
medical_docs = [
    "Hypertension is a condition in which the blood pressure in the arteries is persistently elevated.",
    "Diabetes is a metabolic disease that causes high blood sugar. The hormone insulin moves sugar from the blood into your cells.",
    "Asthma is a respiratory condition marked by spasms in the bronchi of the lungs, causing difficulty in breathing.",
    "A migraine is a headache of varying intensity, often accompanied by nausea and sensitivity to light and sound.",
]

# ✅ STEP 3: Create Embeddings & Vector Store
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document

# Convert raw texts to Document objects
documents = [Document(page_content=doc) for doc in medical_docs]

# Split into chunks
splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=50)
docs = splitter.split_documents(documents)

# Use sentence-transformers for embeddings
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create FAISS vector store
db = FAISS.from_documents(docs, embedding)

# ✅ STEP 4: Setup the LLM using Hugging Face pipeline
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

# Load FLAN-T5 model for question answering
qa_pipeline = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    max_length=256,
    model_kwargs={"temperature": 0}
)

# Wrap pipeline with LangChain LLM
llm = HuggingFacePipeline(pipeline=qa_pipeline)

# ✅ STEP 5: Create the Retrieval-Augmented Generation Chain
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=db.as_retriever(),
    return_source_documents=True
)

# ✅ STEP 6: Ask Medical Questions
query = "What is hypertension?"
result = qa_chain(query)

# ✅ STEP 7: Display the Answer and Source
print("🔍 Question:", query)
print("💬 Answer:", result['result'])
print("\n📚 Source Document(s):")
for doc in result['source_documents']:
    print("-", doc.page_content)


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Device set to use cpu


🔍 Question: What is hypertension?
💬 Answer: a condition in which the blood pressure in the arteries is persistently elevated

📚 Source Document(s):
- Hypertension is a condition in which the blood pressure in the arteries is persistently elevated.
- Diabetes is a metabolic disease that causes high blood sugar. The hormone insulin moves sugar from the blood into your cells.
- Asthma is a respiratory condition marked by spasms in the bronchi of the lungs, causing difficulty in breathing.
- A migraine is a headache of varying intensity, often accompanied by nausea and sensitivity to light and sound.
